In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import os
import fitz
import pytesseract
import io
import PyPDF2
import sys

from pprint import pprint
from PIL import Image
from pdf2image import convert_from_path

import util
import lines
import group
import label
import records
import extract

In [2]:
test_files_dir = ".././test_files/"
test_dfs_dir = test_files_dir + "tesseract_data_frames/"
test_output_dir = test_files_dir + "extracted_indexes/"
test_files = util.list_files(test_files_dir)
test_dfs = util.list_files(test_dfs_dir)
file = test_dfs[2]

indexes_path = "H:\Documents\Indexex\ILO_yearly_indices\\tesseract_data_frames"
indexes_csv = util.list_files(indexes_path, suffix=".csv")
output_dir = "H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes"

display(test_dfs, file)

['.././test_files/tesseract_data_frames/\\LS_index_1920.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1921.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1944.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1976.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986.csv',
 '.././test_files/tesseract_data_frames/\\LS_index_1986_split.csv']

'.././test_files/tesseract_data_frames/\\LS_index_1944.csv'

In [3]:
#pdf_df = util.ocr(file, start_page=1, save_to=".././test_files")
pdf_df = pd.read_csv(file)
lines_df = lines.make_lines_df_from_ocr(pdf_df)
bins_x0, bins_x1, x0_n = group.group_line_starts_ends(lines_df)
borders = lines.make_borders_df(bins_x0, bins_x1)

lines_df

,line_text,artifact_text,x0,y0,x1,y1,page,dx
0,INTERNATIONAL LABOUR OFFICE,,144,342,1899,405,1,1755
1,"LEGISLATIVE SERIES, 1944",,137,1363,1897,1512,1,1760
2,INDEXES,,808,2129,1232,2180,1,424
3,"MONTREAL, CANADA",,719,3137,1314,3181,1,595
4,1946,,967,3207,1072,3248,1,105
...,...,...,...,...,...,...,...,...
4163,Apr. 28 L.D. respecting the organisation of la...,,345,2947,2056,2991,82,1711
4164,agreements; labour courts]. No. 311. (Zb. pp. ...,,608,2993,1881,3039,82,1273
4165,G.O. = Gaceta Oficial de los Estados Unidos de...,,351,3081,1649,3126,82,1298
4166,"Zb. = Zbornik sakona 1 naredaba,",,345,3138,1087,3177,82,742


In [4]:
#rec_df = extract.extract_indexes(pdf_df, save_to=test_output_dir + os.path.basename(file))
rec_df = extract.extract_indexes(pdf_df)
#rec_df.to_csv("LS_index_1944.csv")
rec_df

,country,region,text,page
0,ARGENTINE REPUBLIC,,"June 3 Decree No. 13,943/44, to repeal section...",3
1,ARGENTINE REPUBLIC,,"Sept. 11 Decrees No. 24,335/44 and 24,336/44, ...",3
2,ARGENTINE REPUBLIC,,"Nov. 4 Decree No. 29,830/44, to amend certain ...",3
3,AUSTRALIA,,"Apr. 6 An Act to,amend the Invalid and Old-age...",3
4,AUSTRALIA,,Dec. 21 Amendment of the National Security (In...,3
...,...,...,...,...
1356,YUGOSLAVIA,Croatia |,Mar. 29 L.D. respecting family allowances for ...,82
1357,YUGOSLAVIA,Croatia |,Apr. 28 L.D. respecting the organisation of la...,82
1358,YUGOSLAVIA,Croatia |,G.O. = Gaceta Oficial de los Estados Unidos de...,82
1359,YUGOSLAVIA,Croatia |,"Zb. = Zbornik sakona 1 naredaba,",82


In [9]:
rec_df.loc[rec_df["page"]==5]

,country,region,text,page
22,CANADA,,Jan. 10 O.C. amending the National Selective S...,5
23,CANADA,,"Sept. 1 O.C. amending sections 15, 16 and 48 o...",5
24,CHILE,,"June 23 Act No. 7771, to amend section 1 of Ac...",5
25,DENMARK,,"Mar. 21 Legislative Order No. 93, to continue ...",5
26,DENMARK,,"Apr. 1 Legislative Order No. 110, to amend tem...",5
27,ECUADOR,,"Aug. 8 Decree No. 679, to amend the Social Ins...",5
28,ECUADOR,,"Aug. 9 Decree No. 729, to amend the Labour Cod...",5
29,ECUADOR,,"Aug. 9 Decree No. 771, to provide that section...",5
30,FINLAND,,"Apr. 27 Act to amend the Act of 2nd June, 1939...",5
31,FINLAND,,R.O. = Registro Oficial.,5


In [76]:
df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
df = label.assign_labels(df, x0_n)

ind_df, p_l, p_g = label.correct_x0_types(df, bins_x0, bins_x1, x0_n)
ind_df = label.assign_labels(ind_df, x0_n)
ind_df = label.approve_correction(df, ind_df, p_l)

ind_df["new_label"] = ind_df["label"]
ind_df = label.improve_country_classification(ind_df)

ind_df = records.extract_records(ind_df)

In [77]:
df.loc[df["page"]==9]

,line_text,artifact_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
428,(115) 9 1987—List No. 3,,212,248,2031,300,9,1819,0,2,start
429,BARBADOS (cont.),,209,389,601,429,9,392,0,0,country
430,"Make provisions, inter alia, for survivor’s pe...",[,291,455,1300,508,9,1009,1,0,end
431,25/VII/1985 National Insurance and Social Secu...,,209,527,2027,573,9,1818,0,2,start
432,Regulations 1985. S.I. 1985 No. 124.,,277,575,975,616,9,698,1,0,end
433,Contains provisions for death benefits. ],[,289,639,1051,691,9,762,1,0,end
434,30/X11/1985 Sugar Industry (Sugar Workers Prov...,,209,711,2019,755,9,1810,0,2,start
435,Provides for the imposition of a levy on sugar...,[,286,780,2026,827,9,1740,1,2,middle
436,paid to the National Insurance Board. ],,273,823,1029,876,9,756,1,0,end
437,"27/I11/1986, Factories (Amendment) Act 1986. N...",,211,897,1251,939,9,1040,0,0,country


In [78]:
ind_df.loc[ind_df["page"]==8]

,country,region,text,page
71,AUSTRIA,,1987—List No. 3 8 (114),8
72,AUSTRIA,,20/II1/1969 Federal Act amending the Act of 25...,8
73,AUSTRIA,,25/V1/1986 New amendment to the Act regarding ...,8
74,AUSTRIA,,3/VII/1986 Federal Act of 3 July 1986 amending...,8
75,AUSTRIA,,3/VII/1986 Federal Act amending the Workers’ P...,8
76,BANGLADESH,,13/11/1985 Industrial Relations (Amendment) Or...,8
77,BANGLADESH,,13/11/1985 The Employment of Labour (Standing ...,8
78,BARBADOS,,25/V11/1985 National Insurance and Social Secu...,8


In [5]:
df = extract.extract_dates(rec_df)
df.iloc[120:].head(30)

,country,region,text,page,full_date,year
120,URUGUAY,,"June 20 Act to amend section 11 of Act No. 9,9...",12,June 20,0
121,URUGUAY,,D.O. = Diario Ofictal Part A.,12,,0
122,URUGUAY,,LS. 1932 (Gam. 1). — ®L.S. 1933 (Gam. 1). — *L...,12,,0
123,URUGUAY,,1S. 1937 (Lee. 1). — *L.S. 1934 (Maur. 2).,12,,0
124,URUGUAY,,8ES. 1939 (U.S.A. 1). — *L.S. 1940 (U.S.A. 1)....,12,,0
125,URUGUAY,,"B). ""LS. 1941 (Ur. 2, A). 13 — 1$44—Indexes II...",12,,0
126,URUGUAY,,"(A) LAWS AND ORDERS OF 1944, PUBLISHED IN. THE...",13,,0
127,LATION PASSED DURING THAT YEAR,,The Index gives (1) the titles of the texts co...,13,The Ind,0
128,LATION PASSED DURING THAT YEAR,and social questions in some cases with brief ...,The titles are arranged chronologically under ...,13,,0
129,LATION PASSED DURING THAT YEAR,under the heading “League of Nations”,When a law has been published in the Legislati...,13,,0


In [25]:
li_df = label.assign_types(lines_df, bins_x0, bins_x1, x0_n)
l = label.assign_labels(li_df, x0_n)
l.head(20)

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
0,International Legislative Series,224,236,2041,287,1,1817,0,2,start
1,Labour Office 1987—List No. 3,225,289,2041,324,1,1816,0,2,start
2,LIST OF RECENT LEGISLATION,587,404,1680,456,1,1093,-1,1,other
3,This list indicates the more important enactme...,288,561,2044,603,1,1756,1,2,middle
4,noted in the official gazettes recently receiv...,218,608,1729,651,1,1511,0,1,start
5,"[This cumulative list, arranged in chronologic...",289,655,2044,697,1,1755,1,2,middle
6,1986.],225,703,332,743,1,107,0,0,country
7,ALGERIA,221,837,428,871,1,207,0,0,country
8,1/X/1985 Decree No. 85-243 laying down model r...,225,904,2043,948,1,1818,0,2,start
9,"training. (Journal officiel, 2 Oct.)",286,952,907,995,1,621,1,0,end


In [77]:
cli_df, p_l, p_g = correct_x0_types(l, bins_x0, bins_x1, x0_n)
ll_df = label.assign_labels(cli_df, x0_n)
lla_df = approve_correction(l, ll_df)

lli_df = lla_df.copy()
lli_df["new_label"] = lli_df["label"]
lli_df = label.improve_country_classification(lli_df)

lli_df.head(50)

Width median 1318.1190476190477
[(1645.0, 1), (1619.5, 2), (1267.0625, 7)]
Pages small width [7]
Pages big width [1, 2]


,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label,new_label
0,INTERNATIONAL LABOUR OFFICE,199,254,1790,332,1,1591,0,2,other,other
1,"LEGISLATIVE SERIES, 1921",223,1201,1748,1363,1,1525,1,1,start,start
2,(PARTS I AND ID),695,1404,1292,1469,1,597,-1,0,other,other
3,INDEXES,793,2040,1192,2113,1,399,-1,0,other,other
4,"GENEVA, SWITZERLAND",640,3143,1346,3191,1,706,2,1,end,end
5,Price: 8 d.,145,3238,413,3278,1,268,-1,0,other,other
6,INTERNATIONAL LABOUR OFFICE,465,268,1998,325,2,1533,0,1,other,other
7,"(GENEVA, SWITZERLAND)",908,373,1548,419,2,640,-1,0,other,other
8,"LEGISLATIVE SERIES, 1921",601,967,1842,1053,2,1241,2,1,end,end
9,(PARTS 1 AND II),977,1080,1486,1143,2,509,-1,0,other,other


In [78]:
l.loc[lli_df["page"]==7]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
257,—_—~ 7 ee,899,229,1159,273,7,260,-1,1,other
258,Aug. 19th Act to provide for the reorganisatio...,270,342,1499,380,7,1229,1,2,start
259,regulation of railways and the discharge of,568,389,1496,428,7,928,2,2,middle
260,liabilities arising in connection with the pos...,568,437,1508,474,7,940,2,2,middle
261,"of railways, and otherwise to amend the Law",568,484,1496,522,7,928,2,2,middle
...,...,...,...,...,...,...,...,...,...,...
313,"Dec, 31st The Workmen’s Compensation (Twister’...",260,3023,1492,3061,7,1232,1,2,start
314,"Order, 1921, made by the Secretary of State",564,3071,1495,3108,7,931,2,2,middle
315,under section 8 of the Worknicn’s Compensation,564,3117,1493,3155,7,929,2,2,middle
316,"Act, rg06 (6 Edw. 7, c. 58)*. To. oe. eee eee ...",564,3167,1807,3204,7,1243,2,1,end


In [79]:
ll_df.loc[lli_df["page"]==7]

,line_text,x0,y0,x1,y1,page,dx,x0_type,x1_type,label
257,—_—~ 7 ee,899,229,1159,273,7,260,-1,1,other
258,Aug. 19th Act to provide for the reorganisatio...,270,342,1499,380,7,1229,1,2,start
259,regulation of railways and the discharge of,568,389,1496,428,7,928,2,2,middle
260,liabilities arising in connection with the pos...,568,437,1508,474,7,940,2,2,middle
261,"of railways, and otherwise to amend the Law",568,484,1496,522,7,928,2,2,middle
...,...,...,...,...,...,...,...,...,...,...
313,"Dec, 31st The Workmen’s Compensation (Twister’...",260,3023,1492,3061,7,1232,1,2,start
314,"Order, 1921, made by the Secretary of State",564,3071,1495,3108,7,931,2,2,middle
315,under section 8 of the Worknicn’s Compensation,564,3117,1493,3155,7,929,2,2,middle
316,"Act, rg06 (6 Edw. 7, c. 58)*. To. oe. eee eee ...",564,3167,1807,3204,7,1243,2,1,end


In [80]:
rec_gr = records.group_records(lli_df)
rec = records.extract_records(lli_df)
rec.head(20)

,country,region,text,page
0,ENTERNATIONAL,,Feb. 9th. Convention respecting insurance agai...,3
1,ENTERNATIONAL,,Feb. 14th Convention between France and Belgiu...,3
2,ENTERNATIONAL,,Mar. 4/16th Agreecinent respecting the treatme...,3
3,ENTERNATIONAL,,Oct. 8th Convention between Italy and the Unit...,3
4,ARGENTINE,,June 8th Act respecting matches manufactured’ ...,3
5,AUSTRIA,,Mar. 1oth Act respecting the regulation of emp...,3
6,AUSTRIA,,Mar. 11th Act to amend certain provisions of t...,3
7,AUSTRIA,,Mar. 22nd Act concerning the regulation of agr...,3
8,AUSTRIA,,May 11th Act relating to the contract of servi...,3
9,AUSTRIA,,June rst Act concerning the regulation of empl...,3


In [82]:
rec.loc[rec["page"]==6]

,country,region,text,page
57,FRANCE,,Ort. 30th Decree to amend the Decree of 26th A...,6
58,GERMANY,,Feb. ist Order No. 7968 to prohibit the manufa...,6
59,GERMANY,,"Feb, 2nd Order respecting the painting of ship...",6
60,GERMANY,,Feb. 5th Act respecting the works balance-shee...,6
61,GERMANY,,Apr. 11th Act to amend the law relating to acc...,6
62,GERMANY,,July zoth Act respecting maternity benefit and...,6
63,(BREMEN),,July 17th Act respecting the Council of Wage-c...,6
64,(BREMEN),,July x7th Act respecting theCouncil of Salarie...,6
65,(BREMEN),(Prussia),"Jan, 31st Order providing for (1) the amendmen...",6
66,(BREMEN),(Prussia),july rst Act to vary the rates of contribution...,6


In [18]:
#rec.to_csv("index_1986.csv")

In [44]:
# correct x0_type for pages where something went wrong
def correct_x0_types(lines_df, bins_x0, bins_x1, x0_n):
    df = lines_df.copy()
    start_page = lines_df["page"].min()

    text_widths = list(lines.make_borders_df(bins_x0, bins_x1)["dx"]) # difference between mean of first and last bin for x0 for every page

    tw = text_widths.copy()
    tw.sort()
    width_median = tw[int(len(tw)/2)]
    print("Width median", width_median)

    f = filter(lambda w: (not util.similar_to(w, width_median, 10)), text_widths) # filter widths that differ from the rest
    f = list(f)

    p = []
    for w in range(len(f)):
        p.append((f[w], text_widths.index(f[w]) + start_page)) # add page to strange width
    
    print(p)

    p_l = [a for w, a in p if w<width_median]
    p_g = [a for w, a in p if w>width_median]
    
    print("Pages small width", p_l)
    print("Pages big width", p_g)

    df.loc[df["page"].isin(p_l) & (df["x0_type"]>=0), "x0_type"] +=1 # correct wrong x0_type for p_l

    # correct wrong x0_type (and x1_type) for p_g
    bins = group.get_line_start_end_bins(df.loc[df["page"].isin(p_g)])
    bins_x0_cor = group.get_relevant_x0_bins(bins[0], x0_n, drop_first=True) # drop bin on the very left
    df_cor = label.assign_types(df.loc[df["page"].isin(p_g)], bins_x0_cor, bins_x1.loc[bins_x1["page"].isin(p_g)], x0_n)
    df.loc[df["page"].isin(p_g), ["x0_type", "x1_type"]] = df_cor[["x0_type", "x1_type"]]

    return df, p_l, p_g

In [17]:
for i, file in enumerate(indexes_csv):
    save_path = output_dir + os.sep + os.path.basename(file)
    extract.extract_indexes(pd.read_csv(file), save_to=save_path)

Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1976.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1977.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1978.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1979.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1981.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1982.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_Index_1983.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1983_Spanish.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_yearly_indices\extracted_indexes\LS_index_1984.csv.
Saved extracted indexes to H:\Documents\Indexex\ILO_year

## Probleme
- Dokumente mit 2 Spalten FIXED
- Dokumente mit Länder-Überschrift über ganze Breite der Seite IMPROVED
- Länder- und Regionenspalte vereinen?
    - Regionenerkennung oft nicht richtig, weil Groß- und Kleinbuchstaben vom OCR nicht richtig erkannt werden
- Dokumente/Seiten mit vielen Artifakten
    - 1944
        - S. 8-9, schlechte Ländererkennung FIXED
            - S. 25 falsches Labeling FIXED
        - S. 28 Inhalt fehlt? TESSERACT PROBLEM
        - S. 10-11, schlechte Ländererkennung
    

## Fragen

